<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/TDGClusteringJerarquicoV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hdbscan

     |████████████████████████████████| 6.4MB 7.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311668 sha256=a02476f54f860c5133a2c8c2bf7a83db4bbe0bc7c51956fd40a746f689d942bc
  Stored in directory: /root/.cache/pip/wheels/42/63/fb/314ad6c3b270887a3ecb588b8e5aac50b0fad38ff89bb6dff2
Successfully built hdbscan


In [2]:
!pip install gower

  Created wheel for gower: filename=gower-0.0.5-cp37-none-any.whl size=4232 sha256=d24adc29c147692761d4138c594c8f6ee1750e96fb18e0966f9cfc806034c5d4
  Stored in directory: /root/.cache/pip/wheels/c0/09/9b/072d54d6ced0f43a179852e3f09532d0131e25ff7cb4e5ee75
Successfully built gower


In [3]:
from google.colab import drive
import pandas as pd
import numpy as np
import hdbscan
import gower

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!ls -l /content/gdrive/MyDrive/TDGdata/

total 11917268
-rw------- 1 root root    1108073 Apr 11 21:27 dfDims.csv
-rw------- 1 root root     795978 Apr 11 21:27 dfNoDims.csv
-rw------- 1 root root    2569549 Apr 11 21:27 k2Dims.csv
-rw------- 1 root root    1506267 Apr 11 21:27 k2NoDims.csv
-rw------- 1 root root    2569549 Apr 11 21:27 k3Dims.csv
-rw------- 1 root root    1506267 Apr 11 21:27 k3NoDims.csv
-rw------- 1 root root 5596536100 Apr 11 23:46 scores2.csv
-rw------- 1 root root 6596688400 Apr 11 23:40 scores.csv


In [6]:
# Matriz de distancias (disimilitud)
df = pd.read_csv("/content/gdrive/MyDrive/TDGdata/scores.csv", header = None)

In [7]:
df.iloc[0][0:5]

0    0.000000
1    0.016479
2    0.591425
3    0.000673
4    1.002986
Name: 0, dtype: float64

In [8]:
df.shape

(16244, 16244)

In [9]:
# Clustering jerarquico
clusterer = hdbscan.HDBSCAN(metric='precomputed', min_cluster_size=100)

In [10]:
clusterer.fit(df)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='precomputed', min_cluster_size=100, min_samples=None, p=None,
        prediction_data=False)

In [11]:
np.unique(clusterer.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [12]:
# Dataset de inventarios sin dimensiones de los productos
dataNoDims = pd.read_csv("/content/gdrive/MyDrive/TDGdata/dfNoDims.csv")

In [13]:
dataNoDims.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR,CONTRPERC
0,17,IG56,HTS119,1/1,B,PC,CAT17,-289.8802,-2.63,0.01
1,17,IG56,HTS119,1/1,B,PC,CAT17,289.5527,25.00,0.00
2,17,IG56,HTS119,1/1,C,PC,CAT17,1111.3121,50.00,0.00
3,17,IG56,HTS119,1/1,B,PC,CAT17,-266.8633,-9.09,0.00
4,15,IG56,HTS119,1/1,C,PC,CAT17,-517.0436,-11.11,0.00


In [14]:
dataNoDims.shape

(16244, 10)

In [15]:
# Matriz de distancias de Gower
gowerDistMatrix = gower.gower_matrix(dataNoDims)

In [16]:
gowerDistMatrix = np.float64(gowerDistMatrix)

In [17]:
clusterer2 = hdbscan.HDBSCAN(metric='precomputed', min_cluster_size=100)

In [18]:
clusterer2.fit(gowerDistMatrix)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=False, leaf_size=40,
        match_reference_implementation=False, memory=Memory(location=None),
        metric='precomputed', min_cluster_size=100, min_samples=None, p=None,
        prediction_data=False)

In [19]:
np.unique(clusterer2.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42])